<a href="https://colab.research.google.com/github/Leo-xxx12/bootcamp_Leo_Xu/blob/main/stage04_data_acquisition_and_ingestion_homework_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name:
Date:

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [4]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? False


## Helpers (use or modify)

In [5]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [8]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

/tmp/ipython-input-2430833282.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
[*********************100%***********************]  1 of 1 completed


KeyError: "['Adj Close'] not in index"

In [ ]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

In [10]:
import pandas as pd
import requests
import os
from datetime import datetime
import yfinance as yf

# Configuration
SYMBOL = 'AAPL'

# Load API key from .env (if required)
def load_api_key():
    """Load API key from environment or .env file"""
    try:
        # Try to get from environment first
        api_key = os.getenv('ALPHAVANTAGE_API_KEY')
        if api_key:
            return api_key

        # If not in environment, try to load from .env file
        try:
            with open('.env', 'r') as f:
                for line in f:
                    if line.startswith('ALPHAVANTAGE_API_KEY='):
                        return line.split('=', 1)[1].strip()
        except FileNotFoundError:
            print("No .env file found")

        return None
    except Exception as e:
        print(f"Error loading API key: {e}")
        return None

# Choose ticker/endpoint and determine data source
def get_stock_data(symbol):
    """
    Choose one ticker or endpoint.
    Request data with requests (or yfinance as fallback).
    """

    # Load API key from .env (if required)
    api_key = load_api_key()
    use_alpha = bool(api_key)

    if use_alpha:
        print(f"Using Alpha Vantage API for {symbol}")
        try:
            # Request data with requests
            url = 'https://www.alphavantage.co/query'
            params = {
                'function': 'TIME_SERIES_DAILY_ADJUSTED',
                'symbol': symbol,
                'outputsize': 'compact',
                'apikey': api_key
            }

            r = requests.get(url, params=params, timeout=30)
            r.raise_for_status()
            js = r.json()

            # Check for API errors
            if 'Error Message' in js:
                raise Exception(f"API Error: {js['Error Message']}")
            if 'Note' in js:
                raise Exception(f"API Limit: {js['Note']}")

            # Convert to DataFrame; parse dtypes (dates, floats)
            key = [k for k in js if 'Time Series' in k][0]
            df_api = pd.DataFrame(js[key]).T.reset_index()
            df_api = df_api.rename(columns={
                'index': 'date',
                '5. adjusted close': 'adj_close'
            })

            # Parse dtypes (dates, floats)
            df_api['date'] = pd.to_datetime(df_api['date'])
            numeric_cols = [col for col in df_api.columns if col != 'date']
            for col in numeric_cols:
                df_api[col] = pd.to_numeric(df_api[col], errors='coerce')

            return df_api, 'alpha'

        except Exception as e:
            print(f"Alpha Vantage failed: {e}, falling back to yfinance")
            use_alpha = False

    if not use_alpha:
        print(f"Using yfinance fallback for {symbol}")
        # yfinance as fallback
        df_api = yf.download(symbol, period='3mo', interval='1d').reset_index()

        # Handle column naming more robustly
        print(f"Original columns: {list(df_api.columns)}")
        print(f"Shape: {df_api.shape}")

        # Convert multi-level columns to single level if needed
        if isinstance(df_api.columns, pd.MultiIndex):
            df_api.columns = [col[0] if col[0] != symbol else col[1] for col in df_api.columns]

        # Map existing columns to standardized names
        column_mapping = {}
        current_cols = list(df_api.columns)

        for col in current_cols:
            col_lower = str(col).lower()
            if 'date' in col_lower or col == 'Date':
                column_mapping[col] = 'date'
            elif 'open' in col_lower:
                column_mapping[col] = 'open'
            elif 'high' in col_lower:
                column_mapping[col] = 'high'
            elif 'low' in col_lower:
                column_mapping[col] = 'low'
            elif 'close' in col_lower and 'adj' not in col_lower:
                column_mapping[col] = 'close'
            elif 'adj' in col_lower and 'close' in col_lower:
                column_mapping[col] = 'adj_close'
            elif 'volume' in col_lower:
                column_mapping[col] = 'volume'

        df_api = df_api.rename(columns=column_mapping)

        # Ensure we have the required columns
        if 'adj_close' not in df_api.columns and 'close' in df_api.columns:
            df_api['adj_close'] = df_api['close']

        # Parse dtypes (dates, floats) - yfinance usually handles this automatically
        if 'date' in df_api.columns:
            df_api['date'] = pd.to_datetime(df_api['date'])

        print(f"Final columns: {list(df_api.columns)}")

        return df_api, 'yfinance'

def validate_dataframe(df, required_columns=None):
    """
    Validate (required columns, NA counts, shape).
    """
    if required_columns is None:
        required_columns = ['date', 'adj_close']

    validation_results = {
        'shape': df.shape,
        'required_columns_present': all(col in df.columns for col in required_columns),
        'missing_columns': [col for col in required_columns if col not in df.columns],
        'na_counts': df.isnull().sum().to_dict(),
        'total_na': df.isnull().sum().sum(),
        'date_range': None,
        'valid': True,
        'issues': []
    }

    # Check required columns
    if not validation_results['required_columns_present']:
        validation_results['valid'] = False
        validation_results['issues'].append(f"Missing required columns: {validation_results['missing_columns']}")

    # Check shape
    if df.shape[0] == 0:
        validation_results['valid'] = False
        validation_results['issues'].append("DataFrame is empty")

    # Check date range if date column exists
    if 'date' in df.columns:
        validation_results['date_range'] = {
            'start': df['date'].min(),
            'end': df['date'].max(),
            'days': len(df)
        }

    # Check for excessive NAs
    if validation_results['total_na'] > len(df) * 0.5:  # More than 50% NA
        validation_results['issues'].append("Excessive missing values (>50%)")

    return validation_results

def save_raw_csv(df, prefix='api', source='unknown', symbol=''):
    """
    Save raw CSV to data/raw/.
    """
    # Create directory if it doesn't exist
    os.makedirs('data/raw', exist_ok=True)

    # Generate filename with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"{prefix}_{source}_{symbol}_{timestamp}.csv"
    filepath = os.path.join('data/raw', filename)

    # Sort by date before saving
    if 'date' in df.columns:
        df = df.sort_values('date')

    # Save to CSV
    df.to_csv(filepath, index=False)
    print(f"Raw CSV saved to: {filepath}")

    return filepath

def main():
    """Main function to execute the API pull process"""
    print("=== API Pull Process ===")

    # Get stock data
    df_api, source = get_stock_data(SYMBOL)

    # Validate (required columns, NA counts, shape)
    validation_results = validate_dataframe(df_api, ['date', 'adj_close'])

    print(f"\n=== Validation Results ===")
    print(f"Shape: {validation_results['shape']}")
    print(f"Required columns present: {validation_results['required_columns_present']}")
    print(f"Total NA values: {validation_results['total_na']}")
    print(f"Date range: {validation_results['date_range']}")
    print(f"Valid: {validation_results['valid']}")

    if validation_results['issues']:
        print("Issues found:")
        for issue in validation_results['issues']:
            print(f"  - {issue}")

    if not validation_results['valid']:
        print("Validation failed! Please check the data.")
        return None

    # Save raw CSV to data/raw/
    filepath = save_raw_csv(df_api, prefix='api', source=source, symbol=SYMBOL)

    print(f"\n=== Process Complete ===")
    print(f"Data source: {source}")
    print(f"Records: {len(df_api)}")
    print(f"File saved: {filepath}")

    return df_api, validation_results

# Execute if run directly
if __name__ == "__main__":
    result = main()




/tmp/ipython-input-3148973483.py:89: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(symbol, period='3mo', interval='1d').reset_index()
[*********************100%***********************]  1 of 1 completed

=== API Pull Process ===
No .env file found
Using yfinance fallback for AAPL
Original columns: [('Date', ''), ('Close', 'AAPL'), ('High', 'AAPL'), ('Low', 'AAPL'), ('Open', 'AAPL'), ('Volume', 'AAPL')]
Shape: (64, 6)
Final columns: ['date', 'close', 'high', 'low', 'open', 'volume', 'adj_close']

=== Validation Results ===
Shape: (64, 7)
Required columns present: True
Total NA values: 0
Date range: {'start': Timestamp('2025-05-19 00:00:00'), 'end': Timestamp('2025-08-19 00:00:00'), 'days': 64}
Valid: True


Raw CSV saved to: data/raw/api_yfinance_AAPL_20250819_152713.csv

=== Process Complete ===
Data source: yfinance
Records: 64
File saved: data/raw/api_yfinance_AAPL_20250819_152713.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [ ]:
SCRAPE_URL = 'https://example.com/markets-table'  # TODO: replace with permitted page
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

In [ ]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

In [11]:
import pandas as pd
import requests
import os
from datetime import datetime
from bs4 import BeautifulSoup

def scrape_public_table(scrape_url=None):
    """
    Scrape a Small Table (required):
    - Public, permitted page with a simple table
    - Parse with BeautifulSoup; build DataFrame
    - Validate numeric/text columns
    - Save raw CSV to data/raw/
    """

    # Default to a public table if no URL provided
    if scrape_url is None:
        # Example: Wikipedia table (public, permitted)
        scrape_url = 'https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue'

    print(f"Scraping table from: {scrape_url}")

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

    try:
        # Request the page
        resp = requests.get(scrape_url, headers=headers, timeout=30)
        resp.raise_for_status()

        # Parse with BeautifulSoup; build DataFrame
        soup = BeautifulSoup(resp.text, 'html.parser')

        # Find the first table with data
        tables = soup.find_all('table')

        if not tables:
            raise Exception("No tables found on the page")

        # Try to find a table with meaningful data
        df_scrape = None
        for table in tables:
            rows = [[c.get_text(strip=True) for c in tr.find_all(['th', 'td'])] for tr in table.find_all('tr')]

            # Filter out empty rows
            rows = [r for r in rows if r and any(cell.strip() for cell in r)]

            if len(rows) > 2:  # At least header + 2 data rows
                # Use first row as header
                header = rows[0]
                data = rows[1:]

                # Ensure all rows have same number of columns
                max_cols = len(header)
                data = [row[:max_cols] + [''] * (max_cols - len(row)) for row in data]

                df_scrape = pd.DataFrame(data, columns=header)
                break

        if df_scrape is None or df_scrape.empty:
            raise Exception("Could not extract meaningful table data")

    except Exception as e:
        print(f'Scrape failed, using demo table: {e}')

        # Fallback demo table
        html = '<table><tr><th>Company</th><th>Revenue</th><th>Country</th></tr><tr><td>Walmart</td><td>611.3</td><td>USA</td></tr><tr><td>Amazon</td><td>513.98</td><td>USA</td></tr><tr><td>Apple</td><td>394.33</td><td>USA</td></tr></table>'
        soup = BeautifulSoup(html, 'html.parser')
        rows = [[c.get_text(strip=True) for c in tr.find_all(['th', 'td'])] for tr in soup.find_all('tr')]
        header, *data = [r for r in rows if r]
        df_scrape = pd.DataFrame(data, columns=header)

    print(f"Scraped table shape: {df_scrape.shape}")
    print(f"Columns: {list(df_scrape.columns)}")

    return df_scrape

def validate_scraped_data(df):
    """
    Validate numeric/text columns for scraped data
    """
    validation_results = {
        'shape': df.shape,
        'columns': list(df.columns),
        'column_types': {},
        'numeric_columns': [],
        'text_columns': [],
        'issues': [],
        'valid': True
    }

    for col in df.columns:
        # Try to convert to numeric
        numeric_series = pd.to_numeric(df[col], errors='coerce')
        numeric_count = numeric_series.notna().sum()
        total_count = len(df[col])

        # If more than 70% can be converted to numeric, treat as numeric
        if numeric_count / total_count > 0.7:
            validation_results['column_types'][col] = 'numeric'
            validation_results['numeric_columns'].append(col)
            # Update the column to numeric
            df[col] = numeric_series
        else:
            validation_results['column_types'][col] = 'text'
            validation_results['text_columns'].append(col)

    # Check for issues
    if df.shape[0] == 0:
        validation_results['valid'] = False
        validation_results['issues'].append("DataFrame is empty")

    if df.shape[1] == 0:
        validation_results['valid'] = False
        validation_results['issues'].append("No columns found")

    # Check for excessive missing data
    missing_percentage = df.isnull().sum().sum() / (df.shape[0] * df.shape[1])
    if missing_percentage > 0.5:
        validation_results['issues'].append(f"High missing data: {missing_percentage:.1%}")

    return validation_results

def save_scraped_csv(df, prefix='scrape', site='unknown', table='table'):
    """
    Save raw CSV to data/raw/ for scraped data
    """
    # Create directory if it doesn't exist
    os.makedirs('data/raw', exist_ok=True)

    # Generate filename with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"{prefix}_{site}_{table}_{timestamp}.csv"
    filepath = os.path.join('data/raw', filename)

    # Save to CSV
    df.to_csv(filepath, index=False)
    print(f"Scraped CSV saved to: {filepath}")

    return filepath

def main():
    """Main function to execute web scraping process"""
    print("=== Web Scraping Process ===")

    # Scrape a public table
    df_scrape = scrape_public_table()

    # Validate numeric/text columns
    scrape_validation = validate_scraped_data(df_scrape)

    print(f"\n=== Validation Results ===")
    print(f"Shape: {scrape_validation['shape']}")
    print(f"Columns: {scrape_validation['columns']}")
    print(f"Numeric columns: {scrape_validation['numeric_columns']}")
    print(f"Text columns: {scrape_validation['text_columns']}")
    print(f"Valid: {scrape_validation['valid']}")

    if scrape_validation['issues']:
        print("Issues found:")
        for issue in scrape_validation['issues']:
            print(f"  - {issue}")

    # Save raw CSV to data/raw/
    scrape_filepath = save_scraped_csv(df_scrape, prefix='scrape', site='wikipedia', table='companies')

    print(f"\n=== Process Complete ===")
    print(f"Records: {len(df_scrape)}")
    print(f"File saved: {scrape_filepath}")

    return {
        'scrape_data': df_scrape,
        'scrape_validation': scrape_validation,
        'scrape_file': scrape_filepath
    }

# Execute if run directly
if __name__ == "__main__":
    result = main()

=== Web Scraping Process ===
Scraping table from: https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue
Scraped table shape: (50, 9)
Columns: ['Ranks', 'Name', 'Industry', 'Revenue', 'Profit', 'Employees', 'Headquarters[note 1]', 'State-owned', 'Ref.']

=== Validation Results ===
Shape: (50, 9)
Columns: ['Ranks', 'Name', 'Industry', 'Revenue', 'Profit', 'Employees', 'Headquarters[note 1]', 'State-owned', 'Ref.']
Numeric columns: ['Ranks']
Text columns: ['Name', 'Industry', 'Revenue', 'Profit', 'Employees', 'Headquarters[note 1]', 'State-owned', 'Ref.']
Valid: True
Scraped CSV saved to: data/raw/scrape_wikipedia_companies_20250819_153359.csv

=== Process Complete ===
Records: 50
File saved: data/raw/scrape_wikipedia_companies_20250819_153359.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.